In [37]:
import sys
import nltk
import sklearn
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
 

In [24]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
df = pd.read_table('/content/drive/MyDrive/SMSSpamCollection', header=None, encoding='utf-8')

In [7]:
df.head()

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5572 non-null   object
 1   1       5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [9]:
classes = df[0]
print(classes.value_counts())

ham     4825
spam     747
Name: 0, dtype: int64


In [12]:
encoder = LabelEncoder()
Y = encoder.fit_transform(classes)

print(Y[:10])

[0 0 1 0 0 1 0 0 1 1]


In [13]:
text_messages = df[1]
print(text_messages[:10])

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object


In [14]:
text_messages.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

In [17]:
processed=text_messages.str.lower()
processed = processed.str.replace(r'\s+', ' ')
print(processed)

0       go until jurong point crazy available only in ...
1                                ok lar joking wif u oni 
2       free entry in a wkly comp to win fa cup final ...
3            u dun say so early hor u c already then say 
4       nah i don t think he goes to usf he lives arou...
                              ...                        
5567    this is the nd time we have tried contact u u ...
5568                   will b going to esplanade fr home 
5569    pity was in mood for that so any other suggest...
5570    the guy did some bitching but i acted like i d...
5571                            rofl its true to its name
Name: 1, Length: 5572, dtype: object


In [21]:
stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x: ' '.join(
    term for term in x.split() if term not in stop_words))

In [22]:
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(
    ps.stem(term) for term in x.split()))

In [33]:
processed[0]

'go jurong point crazi avail bugi n great world la e buffet cine got amor wat'

In [25]:
all_words = []

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)

In [28]:
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))

Number of words: 6312
Most common words: [('u', 1228), ('call', 695), ('go', 462), ('get', 458), ('ur', 391), ('gt', 318), ('lt', 316), ('come', 305), ('ok', 293), ('day', 293), ('free', 288), ('know', 275), ('love', 266), ('like', 261), ('time', 254)]


In [29]:
word_features = list(all_words.keys())[:1500]

ValueError: ignored

In [36]:
X_train,X_test,y_train,y_test=train_test_split(processed,Y,test_size=0.2,random_state=1)

In [38]:
CountVec = CountVectorizer(ngram_range=(1,1))

In [39]:
Count_data = CountVec.fit_transform(X_train)

In [48]:
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours
sme = SMOTEENN(random_state=42 , enn=EditedNearestNeighbours(sampling_strategy='all'))
X_resampled,y_resampled = sme.fit_resample(Count_data,y_train)

In [52]:
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(X_resampled,y_resampled)

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [53]:
test_data=CountVec.transform(X_test)
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours
sme = SMOTEENN(random_state=42 , enn=EditedNearestNeighbours(sampling_strategy='all'))
X_re_test,y_re_test = sme.fit_resample(test_data,y_test)
predictions = randomclassifier.predict(X_re_test)

In [55]:
matrix=confusion_matrix(y_re_test,predictions)
print(matrix)
score=accuracy_score(y_re_test,predictions)
print(score)
report=classification_report(y_re_test,predictions)
print(report)

[[144   3]
 [  7 946]]
0.990909090909091
              precision    recall  f1-score   support

           0       0.95      0.98      0.97       147
           1       1.00      0.99      0.99       953

    accuracy                           0.99      1100
   macro avg       0.98      0.99      0.98      1100
weighted avg       0.99      0.99      0.99      1100

